In [ ]:
import pandas as pd
import seaborn as sns
import xlsxwriter

pd.options.display.float_format = '{:,}'.format

data = {'borough':['The Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island'],'population':[42.10,70.82,22.83,108.53,58.37]}

df = pd.DataFrame(data)

df